# Classification task

In [ ]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import collections
from scipy.stats.stats import pearsonr
import pandas as pd
import os
from datetime import date

from scipy.spatial.distance import pdist,  squareform
from tqdm.notebook import tqdm
from sklearn.metrics import silhouette_score, pairwise_distances # For Model evaluation
from sklearn.neighbors import NearestNeighbors

import seaborn as sns
import re

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
#index_col=False say to not use the first column as ID
df_players = pd.read_csv('players.csv', sep=',', index_col=0) 

In [ ]:
df_players.info()

In [ ]:
df_players[['whole_name', 'sex', 'hand', 'best_rank','best_rank_points', 'best_of_3_match', 'best_of_5_match',
           'best_of_3_wins', 'best_of_5_wins', 'w_finals', 'tot_minutes', 'sv1st', 'sv1st_win', 'sv2nd_win', 
           'df', 'ace_perc', 'bpS_perc', 'wmatch', 'lmatch', 'nmatch', 'n_tourney']]

### Decision tree

### Rule based

### Naive Bayes

### SVM

### Bagging

### Boosting

### AdaBoost

### Random Forest

### Neural Networks

### KNN